# Child in Need Plan (CiNP) - Demographic Analysis

In [1]:
#import bigquery
from google.cloud import bigquery
from google.cloud import bigquery_storage
 
#other needed libraries
import os
import pandas as pd
import numpy as np
import pandas_gbq
import seaborn as sns
import matplotlib.pyplot as plt


import warnings
warnings.filterwarnings('ignore')
plt.style.use('tableau-colorblind10')
color='#702A7D'
grey_color = '#A9A9A9'


os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/jupyter/.config/gcloud/application_default_credentials.json"

#Instatiate BigQuery Client
client = bigquery.Client()

In [2]:
cinp_query = """
WITH LatestAssessment AS (
    SELECT
        a.person_id,
        a.StartDate,
        ROW_NUMBER() OVER (PARTITION BY a.person_id ORDER BY a.StartDate DESC) as rn,
        f.category
    FROM
        yhcr-prd-bradfor-bia-core.CB_2649.tbl_bmbc_Childrens_Social_Services_Assessments AS a
    LEFT JOIN
        yhcr-prd-bradfor-bia-core.CB_2649.cb_FactorLookup AS f
        ON a.factorid = f.factorid
)
SELECT
  a.person_id, a.YearOfBirth
  ,a.PCArea, a.EthnicOrigin
  ,a.StartDate, a.EndDate
  ,p.birth_datetime AS DateOfBirth
  ,la.category AS Category
FROM
  yhcr-prd-bradfor-bia-core.CB_2649.cb_bmbc_ChildrensSocialServices_CiNP AS a
  
LEFT JOIN
    yhcr-prd-bradfor-bia-core.CB_2649.person AS p
ON
    a.person_id = p.person_id
LEFT JOIN
    LatestAssessment la
    ON a.person_id = la.person_id
    AND la.rn = 1;
"""

In [3]:
cinp = pandas_gbq.read_gbq(cinp_query)
cinp.info()

Downloading: 100%|██████████|
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5749 entries, 0 to 5748
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype              
---  ------        --------------  -----              
 0   person_id     5749 non-null   object             
 1   YearOfBirth   5749 non-null   Int64              
 2   PCArea        5749 non-null   object             
 3   EthnicOrigin  5749 non-null   object             
 4   StartDate     5749 non-null   dbdate             
 5   EndDate       4322 non-null   dbdate             
 6   DateOfBirth   5733 non-null   datetime64[us, UTC]
 7   Category      5096 non-null   object             
dtypes: Int64(1), datetime64[us, UTC](1), dbdate(2), object(4)
memory usage: 365.1+ KB
